In [65]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import time, locale
from IPython import display as d
import nvd3
nvd3.ipynb.initialize_javascript(use_remote=True)

locale.setlocale(locale.LC_ALL, 'es_ES')
locale.setlocale(locale.LC_ALL, 'C')
locale.format("%10.2f", 1234.567, grouping=True) 
pd.options.display.float_format = '{:,.2f}'.format

# Static files
FILE_SUBVENCIONES = 'data/datos_subvenciones.csv'
FILE_AYUDAS = 'data/ayudas.json'
FILE_ELECCIONES = 'data/04_201105_1.csv'
FILE_MUNICIPIOS = 'data/smex99.csv'

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [66]:
#Read files
subvenciones = pd.read_csv(FILE_SUBVENCIONES, decimal=',')
ayudas = pd.read_json(FILE_AYUDAS)
municipios = pd.read_csv(FILE_MUNICIPIOS, quotechar='"', sep=';', decimal=',', thousands='.')
elecciones = pd.read_csv(FILE_ELECCIONES, quotechar='"', sep=';', decimal=',', thousands='.')

El importe total de las subvenciones includas en los datos abiertos es:

In [67]:
subvenciones['importe'].sum()

1028817129.8299986

In [68]:
subvenciones['mes_concesion'] = subvenciones['fecha_concesion'].apply(lambda x: datetime.fromtimestamp(x).strftime('%Y%m'))
subvenciones['fecha_concesion_tmp'] = subvenciones['fecha_concesion'].apply(lambda x: datetime.fromtimestamp(x).strftime('%Y-%m-%d'))

In [69]:
subvenciones_por_fecha = subvenciones.groupby('fecha_concesion').sum()
subvenciones_por_mes = subvenciones.groupby('mes_concesion').sum()
len(subvenciones_por_mes.index)

15

In [70]:
type = 'stackedAreaChart'
chart2 = nvd3.stackedAreaChart(name=type,height=450,width=450,use_interactive_guideline=True)
#nb_element = 50
#xdata = range(len(list(subvenciones_por_mes.index)))
xdata = list(subvenciones_por_mes.index)
ydata = list(subvenciones_por_mes['importe'])
#ydata = [i * random.randint(1, 10) for i in range(nb_element)]
#ydata2 = [x * 2 for x in ydata]
#ydata3 = [x * 5 for x in ydata]
chart2.add_serie(name="serie 1", y=ydata, x=xdata)
#chart2.add_serie(name="serie 2", y=ydata2, x=xdata)
#chart2.add_serie(name="serie 3", y=ydata3, x=xdata)
chart2

,id,base_reguladora,convocatoria,consejeria,organo_gestor,tipo_subvencion,programa,denominacion_programa,importe,finalidad,id_beneficiario,nombre_beneficiario,fecha_concesion,partida_pres
0,1,ORDEN 5 DE DICIEMBRE DE 2006,ORDEN 5 DICIEMBRE DE 2006 Orden 5 de diciembre...,"C.EDUCACION, CULTURA Y DEPORTE","EDUCACION,CULTURA Y DEPORTE GR",REGLADA,32D,FORM. PROFESIONAL PARA EMPLEO,"134,036.28",FOMENTO DEL EMPLEO,P1800200F,AYUNT.DE AGRON,2014-01-08,NaN
1,2,ORDEN 5 DE DICIEMBRE DE 2006,ORDEN 5 DICIEMBRE DE 2006 Orden 5 de diciembre...,"C.EDUCACION, CULTURA Y DEPORTE","EDUCACION,CULTURA Y DEPORTE GR",REGLADA,32D,FORM. PROFESIONAL PARA EMPLEO,"134,036.28",FOMENTO DEL EMPLEO,P1819800B,AYUNT.DE ZUJAR,2014-01-09,NaN
2,3,ORDEN 5 DE DICIEMBRE DE 2006,ORDEN 5 DICIEMBRE DE 2006 Orden 5 de diciembre...,"C.EDUCACION, CULTURA Y DEPORTE","EDUCACION,CULTURA Y DEPORTE GR",REGLADA,32D,FORM. PROFESIONAL PARA EMPLEO,"134,036.28",FOMENTO DEL EMPLEO,G18792234,ASOC ECUESTRE ENTRE LA VEGA Y,2014-01-09,NaN
3,4,ORDEN 5 DE DICIEMBRE DE 2006,ORDEN 5 DICIEMBRE DE 2006 Orden 5 de diciembre...,"C.EDUCACION, CULTURA Y DEPORTE","EDUCACION,CULTURA Y DEPORTE GR",REGLADA,32D,FORM. PROFESIONAL PARA EMPLEO,"134,036.28",FOMENTO DEL EMPLEO,P1818700E,AYUNT.DE VALOR,2014-01-09,NaN
4,5,ORDEN 5 DE DICIEMBRE DE 2006,ORDEN 5 DICIEMBRE DE 2006 Orden 5 de diciembre...,"C.EDUCACION, CULTURA Y DEPORTE","EDUCACION,CULTURA Y DEPORTE GR",REGLADA,32D,FORM. PROFESIONAL PARA EMPLEO,"81,854.28",FOMENTO DEL EMPLEO,P1817700F,AYUNT.DE SANTA CRUZ DEL COMERC,2014-01-09,NaN


In [199]:
elecciones['Nombre de Comunidad'] = elecciones['Nombre de Comunidad'].apply(lambda x: x.rstrip())
filas_andalucia = elecciones['Nombre de Comunidad'] == 'Andaluc\xc3\xada' #Andalucía
cols_elecciones_significativas = (elecciones[rows_andalucia].sum() > 1000)
resumen_elecciones_andalucia = elecciones[filas_andalucia][elecciones.columns[cols_elecciones_significativas]]

,Nombre de Comunidad,Código de Provincia,Nombre de Provincia,Código de Municipio,Nombre de Municipio,Población,Número de mesas,Total censo electoral,Total votantes,Votos válidos,...,A.VE.MA.,AM+LOS VERDES,OLIVO-I,ULT,P.I.BA -2000,U.D.J,U.P.LA LINEA,P.S.I.R.V.,I.P.J,TPS
0,Andalucía,4,Almería,1,Abla ...,1463,2,1147,932,925,...,0,0,0,0,0,0,0,0,0,0
1,Andalucía,4,Almería,2,Abrucena ...,1367,2,1166,1025,1017,...,0,0,0,0,0,0,0,0,0,0
2,Andalucía,4,Almería,3,Adra ...,24512,31,17363,11095,10964,...,0,0,0,0,0,0,0,0,0,0
3,Andalucía,4,Almería,4,Albánchez ...,814,1,632,506,504,...,0,0,0,0,0,0,0,0,0,0
4,Andalucía,4,Almería,5,Alboloduy ...,667,1,653,595,588,...,0,0,0,0,0,0,0,0,0,0
5,Andalucía,4,Almería,6,Albox ...,11042,14,7757,5553,5515,...,0,0,0,0,0,0,0,0,0,0
6,Andalucía,4,Almería,7,Alcolea ...,902,1,800,677,673,...,0,0,0,0,0,0,0,0,0,0
7,Andalucía,4,Almería,8,Alcóntar ...,598,2,546,492,488,...,0,0,0,0,0,0,0,0,0,0
8,Andalucía,4,Almería,9,Alcudia de Monteagud ...,144,1,161,150,148,...,0,0,0,0,0,0,0,0,0,0
9,Andalucía,4,Almería,10,Alhabia ...,718,1,616,525,524,...,0,0,0,0,0,0,0,0,0,0


Código de Provincia            True
Nombre de Provincia            True
Código de Municipio            True
Nombre de Municipio            True
Población                      True
Número de mesas                True
Total censo electoral          True
Total votantes                 True
Votos válidos                  True
Papeletas a candidaturas       True
Votos en blanco                True
Votos nulos                    True
P.P.                           True
P.S.O.E.                       True
I.U.                           True
CiU                           False
UPyD                           True
E.A.J.-P.N.V.                 False
BILDU-EA-ALTERNATIBA          False
ESQUERRA-AM                   False
B.N.G.                        False
ICV-EUiA-E                    False
PA                             True
CC-PNC-CCN                    False
BLOC-C.M. COMPROMÍS           False
FAC                           False
U.P.N.                        False
PAR                         

In [8]:
from nvd3 import pieChart
type = 'pieChart'
chart = pieChart(name=type, color_category='category20c', height=450, width=450)
xdata = ["Orange", "Banana", "Pear", "Kiwi", "Apple", "Strawberry", "Pineapple"]
ydata = [3, 4, 0, 1, 5, 7, 3]
extra_serie = {"tooltip": {"y_start": "", "y_end": " cal"}}
chart.add_serie(y=ydata, x=xdata, extra=extra_serie)
chart.buildcontent()
chart